In [55]:
import pandas as pd
import numpy as np

In [57]:
valeur = pd.read_csv(r"valeursfoncieres-2023.txt", delimiter="|")

C:\Users\dany7\AppData\Local\Temp\ipykernel_31380\3540214829.py:1: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  valeur = pd.read_csv(r"valeursfoncieres-2023.txt", delimiter="|")


In [58]:
print(valeur.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 43 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Identifiant de document     float64
 1   Reference document          float64
 2   1 Articles CGI              float64
 3   2 Articles CGI              float64
 4   3 Articles CGI              float64
 5   4 Articles CGI              float64
 6   5 Articles CGI              float64
 7   No disposition              int64  
 8   Date mutation               object 
 9   Nature mutation             object 
 10  Valeur fonciere             object 
 11  No voie                     float64
 12  B/T/Q                       object 
 13  Type de voie                object 
 14  Code voie                   object 
 15  Voie                        object 
 16  Code postal                 float64
 17  Commune                     object 
 18  Code departement            object 
 19  Code commune         

In [59]:
valeur_in_cleaning = (valeur.copy())
valeur_in_cleaning.dropna(axis=1, how="all", inplace=True)
print(valeur_in_cleaning.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 35 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   No disposition              int64  
 1   Date mutation               object 
 2   Nature mutation             object 
 3   Valeur fonciere             object 
 4   No voie                     float64
 5   B/T/Q                       object 
 6   Type de voie                object 
 7   Code voie                   object 
 8   Voie                        object 
 9   Code postal                 float64
 10  Commune                     object 
 11  Code departement            object 
 12  Code commune                int64  
 13  Prefixe de section          float64
 14  Section                     object 
 15  No plan                     int64  
 16  No Volume                   object 
 17  1er lot                     object 
 18  Surface Carrez du 1er lot   object 
 19  2eme lot             


#list_surface_carrez_lot = ["Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]
#valeur_in_cleaning["Surface Carrez Lots Total"] = 0
#print(valeur_in_cleaning[[list_surface_carrez_lot[0]]])

for i  in list_surface_carrez_lot:
    valeur_in_cleaning[valeur_in_cleaning[i].isna()] = 0

valeur_in_cleaning.convert_dtypes(convert_floating=True)

print(valeur_in_cleaning.info())

for i  in list_surface_carrez_lot:
    valeur_in_cleaning["Surface Carrez Lots Total"] = valeur_in_cleaning["Surface Carrez Lots Total"] + temp_2
print(valeur_in_cleaning[list_surface_carrez_lot])

#print(valeur_in_cleaning["Surface Carrez Lots Total"])
#valeur_in_cleaning["Surface Carrez Lots Total"] =  valeur_in_cleaning[["Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]].sum()

On regarde maintenenant les colonnes qui qui possèdes des valeur et que l'on veut garder
Pour cela, on va regarder les colonnes restanes et voir le pour centage de ligne remplis
On récupère tout les colonnes dont moint de 10 % des lignes sont remplis. On va aussi regarder si une colonne comprenant la somme des entrées dans les colonnes sur las surface carrés des lots.

In [60]:


index_restant = list(valeur_in_cleaning.columns)
print(index_restant)

insignificante_column = list()




for i in index_restant:
    percentage_use = len((valeur_in_cleaning[i])[valeur_in_cleaning[i].notna()])/len(valeur_in_cleaning[i])
    print(f"Pour {i}, le pourcentage de ligne remplis est de {percentage_use}")
    if percentage_use < 0.1:
        insignificante_column.append(i)



mask = valeur_in_cleaning["Surface Carrez du 1er lot"].notna() | valeur_in_cleaning["Surface Carrez du 2eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 3eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 4eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 5eme lot"].notna()

nb_nn = len(valeur_in_cleaning[mask]["Surface Carrez du 4eme lot"])
print(f"Pour la somme des surface carrez des metros, le pourcentage de ligne remplis est de {nb_nn/valeur_in_cleaning.shape[0]}")

['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere', 'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune', 'Prefixe de section', 'Section', 'No plan', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Nombre de lots', 'Code type local', 'Type local', 'Surface reelle bati', 'Nombre pieces principales', 'Nature culture', 'Nature culture speciale', 'Surface terrain']
Pour No disposition, le pourcentage de ligne remplis est de 1.0
Pour Date mutation, le pourcentage de ligne remplis est de 1.0
Pour Nature mutation, le pourcentage de ligne remplis est de 1.0
Pour Valeur fonciere, le pourcentage de ligne remplis est de 0.9913123271903311
Pour No voie, le pourcentage de ligne remplis est de 0.6201649850214851
Pour B/T/Q, le pourcentage de l

On voit donc q'une colonne contennat la somme des surfac ecarrés de chaque lots serait remplit à 11,9 % donc on vas crée un nouvelle colonnes.
Pour cela on vas d'abord convertir c'est colonne en float :
 - On remplace les NaN par des 0
 - On convertit  les colonnes en entier
 - On fait la somme qui est stocké dans une nouvelle colonne

AttributeError: 'DataFrame' object has no attribute 'to_numeric'

On enlève toutes les données dont le pourcentage est inférieur à 10 %.

In [61]:
#print(valeur_in_cleaning.drop(labels=insignificante_column, axis=1))
print("On va enlever : ")
for i in insignificante_column:
    print(f"- {i}")

valeur_in_cleaning.drop(labels=insignificante_column, axis=1, inplace=True) # étant donner que nous suprimons reelement des collon du au inplace
# en cas d'erreur, recherger les instruction précédente 

On va enlever : 
- B/T/Q
- Prefixe de section
- No Volume
- Surface Carrez du 1er lot
- 2eme lot
- Surface Carrez du 2eme lot
- 3eme lot
- Surface Carrez du 3eme lot
- 4eme lot
- Surface Carrez du 4eme lot
- 5eme lot
- Surface Carrez du 5eme lot
- Nature culture speciale


Sure les lignes restantes, nous allons enlever tous les numéros d'identifications

In [62]:
print((valeur_in_cleaning.drop(labels=["No disposition", "No voie", "Voie",  "No plan"], axis=1)).info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 18 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date mutation              object 
 1   Nature mutation            object 
 2   Valeur fonciere            object 
 3   Type de voie               object 
 4   Code voie                  object 
 5   Code postal                float64
 6   Commune                    object 
 7   Code departement           object 
 8   Code commune               int64  
 9   Section                    object 
 10  1er lot                    object 
 11  Nombre de lots             int64  
 12  Code type local            float64
 13  Type local                 object 
 14  Surface reelle bati        float64
 15  Nombre pieces principales  float64
 16  Nature culture             object 
 17  Surface terrain            float64
dtypes: float64(5), int64(2), object(11)
memory usage: 511.8+ MB
None


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   No disposition              0 non-null      int64  
 1   Date mutation               0 non-null      object 
 2   Nature mutation             0 non-null      object 
 3   Valeur fonciere             0 non-null      object 
 4   No voie                     0 non-null      float64
 5   B/T/Q                       0 non-null      object 
 6   Type de voie                0 non-null      object 
 7   Code voie                   0 non-null      object 
 8   Voie                        0 non-null      object 
 9   Code postal                 0 non-null      float64
 10  Commune                     0 non-null      object 
 11  Code departement            0 non-null      object 
 12  Code commune                0 non-null      int64  
 13  Prefixe de section          0 non-null      float64


C:\Users\q5685\AppData\Local\Temp\ipykernel_21180\3600780839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valeur_in_cleaning.dropna(axis=0, how="any", inplace=True)


In [70]:

a_convertir=['No disposition','Valeur fonciere','Code voie','Code postal','No plan','Nombre de lots','Code type local','Surface reelle bati','Nombre pieces principales','Surface terrain','Code commune']

valeur_in_cleaning[a_convertir] = valeur_in_cleaning[a_convertir].replace(',', '.', regex=True, )
valeur_in_cleaning[a_convertir] = pd.to_numeric(list(valeur_in_cleaning[a_convertir]), errors='coerce')
valeur_in_cleaning[a_convertir] = valeur_in_cleaning[a_convertir].astype(float) # changer le astype(int) si vous volez changer les chifre en int, n'oublier pas de recherger a nouveau tout le document

print(valeur_in_cleaning.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 22 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   No disposition             float64
 1   Date mutation              object 
 2   Nature mutation            object 
 3   Valeur fonciere            float64
 4   No voie                    float64
 5   Type de voie               object 
 6   Code voie                  float64
 7   Voie                       object 
 8   Code postal                float64
 9   Commune                    object 
 10  Code departement           object 
 11  Code commune               float64
 12  Section                    object 
 13  No plan                    float64
 14  1er lot                    object 
 15  Nombre de lots             float64
 16  Code type local            float64
 17  Type local                 object 
 18  Surface reelle bati        float64
 19  Nombre pieces principales  float64
 20  Na